In [1]:
import sqlalchemy
import json
import pandas as pd

with open("./db_config.json", "r") as f:
    config = json.load(f)

USER = config["DB_USER"]
PW = config["DB_PASSWORD"]
HOST = config["DB_HOST"]
PORT = config["DB_PORT"]
DB = config["DB_NAME"]

In [2]:
# DB 연결
engine = sqlalchemy.create_engine(f"mysql+pymysql://{USER}:{PW}@{HOST}:{PORT}/{DB}")

with engine.connect() as conn:
    result = conn.execute(sqlalchemy.text("SELECT VERSION();"))
    print("Connect Success. Version is", result.fetchone())

Connect Success. Version is ('8.4.5',)


In [3]:
import pandas as pd
df_2023_accident = pd.read_csv("./dataset/2023년_스포츠경기장주변_교통사고_transformed.csv")
df_2024_accident = pd.read_csv("./dataset/2024년_스포츠경기장주변_교통사고_transformed.csv")

df_accident_merged = pd.concat([df_2023_accident, df_2024_accident], join='inner', ignore_index=True)

df_accident_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34032 entries, 0 to 34031
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      34032 non-null  object
 1   날짜      34032 non-null  object
 2   사고      34032 non-null  int64 
 3   사망      34032 non-null  int64 
 4   부상      34032 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.3+ MB


In [4]:
df_accident_merged = df_accident_merged.rename(columns={
    "지역": "region",
    "날짜": "accident_date",
    "사고": "accident_count",
    "사망": "death_count",
    "부상": "injury_count"
})

df_accident_merged.head(5)

,region,accident_date,accident_count,death_count,injury_count
0,강원 강릉시,2023.01.01,5,0,10
1,강원 강릉시,2023.01.02,2,0,2
2,강원 강릉시,2023.01.03,3,0,4
3,강원 강릉시,2023.01.04,3,0,4
4,강원 강릉시,2023.01.05,1,0,1


In [5]:
df_accident_merged['accident_date'] = pd.to_datetime(df_accident_merged['accident_date'], format="%Y.%m.%d").dt.date#
df_accident_merged.head(5)                                        

,region,accident_date,accident_count,death_count,injury_count
0,강원 강릉시,2023-01-01,5,0,10
1,강원 강릉시,2023-01-02,2,0,2
2,강원 강릉시,2023-01-03,3,0,4
3,강원 강릉시,2023-01-04,3,0,4
4,강원 강릉시,2023-01-05,1,0,1


In [6]:
# stadium_code 설정
stadium_code_map = {
    "안산상록수체육관": "AS01",
    "안양 정관장 아레나": "AY01",
    "부산 아시아드 경기장": "BS01",
    "사직야구장": "BS02",
    "부산사직체육관": "BS03",
    "천안 종합 운동장": "CA01",
    "천안유관순체육관": "CA02",
    "춘천 송암 스포츠타운": "CC01",
    "청주야구장": "CJ01",
    "창원NC파크": "CW01",
    "창원체육관": "CW02",
    "대구iM뱅크PARK": "DG01",
    "대구 스타디움": "DG02",
    "대구 삼성 라이온즈 파크": "DG03",
    "대구체육관": "DG04",
    "대전 월드컵 경기장": "DJ01",
    "대전 한화생명 볼파크": "DJ02",
    "한화생명 이글스 파크": "DJ03",
    "대전충무체육관": "DJ04",
    "김천 종합 운동장": "GC01",
    "김천실내체육관": "GC02",
    "광주 축구 전용경기장": "GJ01",
    "광주-기아 챔피언스필드": "GJ02",
    "구미박정희체육관": "GM01",
    "강릉 종합 운동장": "GN01",
    "김포솔터축구장": "GP01",
    "군산월명체육관": "GS01",
    "고양 소노 아레나": "GY01",
    "페퍼스타디움": "HS01",
    "화성종합실내체육관": "HS02",
    "인천 축구 전용경기장": "IC01",
    "인천 SSG 랜더스 필드": "IC02",
    "인천계양체육관": "IC03",
    "인천삼산월드체육관": "IC04",
    "두산베어스파크": "IH01",
    "제주 월드컵 경기장": "JE01",
    "전주 월드컵 경기장": "JJ01",
    "전주실내체육관": "JJ02",
    "포항 스틸야드": "PH01",
    "포항야구장": "PH02",
    "서울 월드컵 경기장": "SO01",
    "상암 보조경기장": "SO02",
    "목동 종합 운동장": "SO03",
    "고척 스카이돔": "SO04",
    "잠실야구장": "SO05",
    "잠실실내체육관": "SO06",
    "잠실학생체육관": "SO07",
    "서울장충체육관": "SO08",
    "수원 월드컵 경기장": "SU01",
    "수원 종합 운동장": "SU02",
    "수원 케이티위즈 파크": "SU03",
    "수원 KT 아레나": "SU04",
    "수원체육관": "SU05",
    "의정부체육관": "UJ01",
    "경민대학교 기념관(체육관)": "UJ02",
    "울산 문수 축구경기장": "US01",
    "울산 종합 운동장": "US02",
    "울산문수야구장": "US03",
    "울산동천체육관": "US04",
    "원주종합체육관": "WJ01"
}

In [7]:
df_accident_merged['region'].unique()

array(['강원 강릉시', '강원 원주시', '강원 춘천시', '경기 수원시', '경기 안산시', '경기 의정부시',
       '경기 이천시', '경기 화성시', '경북 구미시', '경북 김천시', '경북 포항시', '광주 광산구',
       '광주 남구', '광주 동구', '광주 북구', '광주 서구', '대구 남구', '대구 동구', '대구 북구',
       '대구 서구', '대구 수성구', '대구 중구', '대전 동구', '대전 서구', '대전 유성구', '대전 중구',
       '부산 남구', '부산 동구', '부산 동래구', '부산 북구', '부산 서구', '부산 연제구', '부산 중구',
       '서울 마포구', '서울 송파구', '서울 양천구', '서울 중구', '울산 남구', '울산 동구', '울산 북구',
       '울산 중구', '인천 계양구', '인천 남동구', '인천 동구', '인천 미추홀구', '인천 서구', '인천 중구',
       '전북 군산시', '전북 전주시', '제주 제주시', '충남 천안시', '충북 청주시'], dtype=object)

In [8]:
# stadium_code 설정
stadium_code_dict = {
    "경기 안산시": ["AS01"],
    "안양": ["AY01"],
    "부산 남구": ["BS01", "BS02", "BS03"],
    "부산 동구": ["BS01", "BS02", "BS03"],
    "부산 동래구": ["BS01", "BS02", "BS03"],
    "부산 북구": ["BS01", "BS02", "BS03"],
    "부산 서구": ["BS01", "BS02", "BS03"],
    "부산 연제구": ["BS01", "BS02", "BS03"],
    "부산 중구": ["BS01", "BS02", "BS03"],
    "충남 천안시": ["CA01", "CA02"],
    "강원 춘천시": ["CC01"],
    "충북 청주시": ["CJ01"],
    "창원": ["CW01", "CW02"],
    "대구 남구": ["DG01", "DG02", "DG03", "DG04"],
    "대구 동구": ["DG01", "DG02", "DG03", "DG04"],
    "대구 북구": ["DG01", "DG02", "DG03", "DG04"],
    "대구 서구": ["DG01", "DG02", "DG03", "DG04"],
    "대구 수성구": ["DG01", "DG02", "DG03", "DG04"],
    "대구 중구": ["DG01", "DG02", "DG03", "DG04"],
    "대전 동구": ["DJ01", "DJ02", "DJ03", "DJ04"],
    "대전 서구": ["DJ01", "DJ02", "DJ03", "DJ04"],
    "대전 유성구": ["DJ01", "DJ02", "DJ03", "DJ04"],
    "대전 중구": ["DJ01", "DJ02", "DJ03", "DJ04"],
    "경북 김천시": ["GC01", "GC02"],
    "광주 광산구": ["GJ01", "GJ02"],
    "광주 남구": ["GJ01", "GJ02"],
    "광주 동구": ["GJ01", "GJ02"],
    "광주 북구": ["GJ01", "GJ02"],
    "광주 서구": ["GJ01", "GJ02"],
    "경북 구미시": ["GM01"],
    "강원 강릉시": ["GN01"],
    "김포": ["GP01"],
    "전북 군산시": ["GS01"],
    "고양": ["GY01"],
    "경기 화성시": ["HS01", "HS02"],
    "인천 계양구": ["IC01", "IC02", "IC03", "IC04"],
    "인천 남동구": ["IC01", "IC02", "IC03", "IC04"],
    "인천 동구": ["IC01", "IC02", "IC03", "IC04"],
    "인천 미추홀구": ["IC01", "IC02", "IC03", "IC04"],
    "인천 서구": ["IC01", "IC02", "IC03", "IC04"],
    "인천 중구": ["IC01", "IC02", "IC03", "IC04"],
    "경기 이천시": ["IH01"],
    "제주 제주시": ["JE01"],
    "전북 전주시": ["JJ01", "JJ02"],
    "경북 포항시": ["PH01", "PH02"],
    "서울 마포구": ["SO01", "SO02", "SO03", "SO04", "SO05", "SO06", "SO07", "SO08"],
    "서울 송파구": ["SO01", "SO02", "SO03", "SO04", "SO05", "SO06", "SO07", "SO08"],
    "서울 양천구": ["SO01", "SO02", "SO03", "SO04", "SO05", "SO06", "SO07", "SO08"],
    "서울 중구": ["SO01", "SO02", "SO03", "SO04", "SO05", "SO06", "SO07", "SO08"],
    "경기 수원시": ["SU01", "SU02", "SU03", "SU04", "SU05"],
    "경기 의정부시": ["UJ01", "UJ02"],
    "울산 남구": ["US01", "US02", "US03", "US04"],
    "울산 동구": ["US01", "US02", "US03", "US04"],
    "울산 북구": ["US01", "US02", "US03", "US04"],
    "울산 중구": ["US01", "US02", "US03", "US04"],
    "강원 원주시": ["WJ01"]
}
# '지점명' 컬럼을 기준으로 stadium_code_dict에서 첫 번째 stadium_code 저장
df_accident_merged['stadium_code'] = df_accident_merged['region'].map(lambda x: stadium_code_dict.get(x, [None])[0])

# 매핑되지 않은 데이터(None) 확인
none_count = df_accident_merged['stadium_code'].isnull().sum()
print(f"매핑되지 않은 데이터 수: {none_count}")

if none_count > 0:
    print(df_accident_merged[df_accident_merged['stadium_code'].isnull()][['region', 'stadium_code']])

매핑되지 않은 데이터 수: 0


In [10]:
columns_to_insert = ["region", "accident_date", "accident_count", "death_count", "injury_count", "stadium_code"]

df_accident_merged[columns_to_insert].to_sql(
    name='traffic_accident',
    con=engine,
    if_exists='append',  # append: 기존 데이터 유지하고 추가
    index=False,         # DataFrame의 인덱스는 사용하지 않음
    dtype={
        "region": sqlalchemy.String(20),
        "accident_date": sqlalchemy.Date,
        "accident_count": sqlalchemy.Integer,
        "death_count": sqlalchemy.Integer,
        "injury_count": sqlalchemy.Integer,
        "stadium_code": sqlalchemy.String(10)
    }
)
#        를 넣기 위한 딕셔너리를 짜야함.

34032